### Plotting time behaviour of different projects

In [216]:
import pandas as pd

In [251]:
df = pd.read_csv('mv_port.csv')

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (1,2,3,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,40,41,45,51,58,59,61,64) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [252]:
df.head()

,Unnamed: 0,Project Number_x,Project Name,Health and Safety Status,Project Status,SalesCloudClientNo,Contract Type Name,Ultimate Customer AUS Market Sector,Division Short Name,location_2,...,Project Number_y,Project Start Date,POC,Contract Status,Related Opportunity Project Number_y,Project Manager Employee Number (Current),Project Progress Status (Current),Scheduled End Date,Date,Project Number
0,0,unspecified,UNIVAR - PROJECT MONARCH,NaN,Open/Active,10239939.0,Time and Material,NaN,Environment,RE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201601,unspecified
1,1,30001238,YJWWTP Post-Sandy Rehab- PhIII,Plan Required,Open/Active,10238060.0,Cost plus Fixed Fee,Municipalities/Local Govt.,Water,NT,...,NaN,2014-12-21 00:00:00,NaN,NaN,NaN,24232.0,NaN,2021-05-03,201601,30001238
2,2,30001347,Abandoned Chemical Sales Fac.,Plan Required,Open/Active,10238193.0,D,State/Regional Government,Water,NT,...,NaN,2012-04-27 00:00:00,NaN,NaN,NaN,24986.0,NaN,2020-10-01,201601,30001347
3,3,30001355,NYSDEC Jimmy's RA,Plan Required,Open/Active,10238193.0,Cost plus Fixed Fee,State/Regional Government,Environment,RE,...,NaN,2013-05-19 00:00:00,NaN,NaN,NaN,24341.0,NaN,2021-01-11,201601,30001355
4,4,30001367,Chemical Sales RD WA 34,Plan Required,Open/Active,10238193.0,D,State/Regional Government,Water,NT,...,NaN,2015-07-01 00:00:00,NaN,NaN,NaN,24986.0,NaN,2020-06-11,201601,30001367


In [253]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df['Project Number'] = df['Project Number'].astype('str') 

### for now lets exclude the 2021-04 observation

In [254]:
df = df[df['Date'] != 202104]

### Total Project duration

In [255]:
df['Projectduration'] = df.groupby(['Project Number'])['Date'].transform('count')

In [256]:
df[df['Project Number'] == '30001798']['Projectduration']

2208       63
2461       63
4894       63
7428       63
10055      63
           ..
933487     63
972070     63
1009374    63
1038580    63
1079964    63
Name: Projectduration, Length: 63, dtype: int64

## Let's alculate cumsumNR

In [257]:
df['Actual Net Revenue - MTD'].fillna(0, inplace=True)

df['ActualNR_sum'] = df.groupby("Project Number")['Actual Net Revenue - MTD'].cumsum()

In [258]:
df[df['Project Number'] == '30001798']['ActualNR_sum']

2208       10223.97
2461       12507.97
4894       13099.24
7428       14328.92
10055      16406.67
             ...   
933487     44090.87
972070     45126.27
1009374    45126.27
1038580    45126.27
1079964    45306.27
Name: ActualNR_sum, Length: 63, dtype: float64

### We need to drop some columns

In [259]:
#df.pop('CVP Code')
df.pop('interest_current.1')
df.pop('Total WIP')
df.pop('Positive WIP')
df.pop('Project Number_y')
df.pop('ID')
df.pop('POC')
df.pop('Contract Status')
df.pop('Related Opportunity Project Number_y')
df.pop('Project Progress Status (Current)')

0                  NaN
1                  NaN
2                  NaN
3                  NaN
4                  NaN
              ...     
1269166    Unspecified
1269167    Unspecified
1269168    Unspecified
1269169    Unspecified
1269170    Unspecified
Name: Project Progress Status (Current), Length: 1269171, dtype: object

### forward fill funding amount

In [260]:
def obtain_df(project, df):
    df_project = df[df['Project Number'] == project]
    #df_project = df_project[['Fiscal Period Code', 'Funding Amount - ITC (Snapshot)', 'Backlog NR', 'Actual Gross Revenue - MTD']].copy()
    df_project['Backlog_Shifted'] = df_project['Backlog NR'].transform(lambda x: x.shift())
    df_project.reset_index(inplace=True)
    df_project = df_project.loc[:, ~df_project.columns.str.contains('^index')]
    return df_project

def random_project(df):
    df = df.sample(n=1)
    project_number = df['Project Number'].item()
    return project_number

### Let's sort the dataframe by project number and only run this preprocessing for a few projects now

In [261]:
df.sort_values(by=['Project Number', 'Date'], inplace=True)
df.reset_index(inplace=True)

### I will now generate 10 smaller dataframes so that the remove NA can be applied

In [127]:
1269171/10

126917.1

In [190]:
#in this function we need to put in the start and preliminary end date, and then it will make sure that we don't exclude a project
def the_splits(start,prelimend):
    dfmutable = df.iloc[start:prelimend]
    x=dfmutable.iloc[-1:]['Project Number']
    y = x.iloc[0]
    lastproject=df[df['Project Number'] == y]
    lastindex=lastproject.index[-1]
    sub_df = df.iloc[:lastindex]
    return sub_df

In [154]:
(1269171-126917)

1142254

In [172]:
start= df1.index[-1] +1
end = start + 126917
df2 = the_splits(start, end)

In [173]:
start= df2.index[-1] +1
end = start + 126917
df3 = the_splits(start, end)

In [174]:
start= df3.index[-1] +1
end = start + 126917
df4 = the_splits(start, end)

In [175]:
start= df4.index[-1] +1
end = start + 126917
df5 = the_splits(start, end)

In [176]:
start= df5.index[-1] +1
end = start + 126917
df6 = the_splits(start, end)

In [177]:
start= df6.index[-1] +1
end = start + 126917
df7 = the_splits(start, end)

In [178]:
start= df7.index[-1] +1
end = start + 126917
df8 = the_splits(start, end)

In [179]:
start= df8.index[-1] +1
end = start + 126917
df9 = the_splits(start, end)

In [184]:
start= df9.index[-1] +1
end = len(df)
df10 = df.iloc[start:end]

In [191]:
#chuncks
def obtain_chucklist(dataFrame):
    chuncks = dataFrame['Project Number'].unique()
    chuncks = chuncks.tolist()
    return chuncks

In [187]:
type(chuncks)

list

In [192]:
def remove_NA(chuncks, dataframe):
    appended_data = []
    
    for project in chuncks:
        df_sub = obtain_df(project, dataframe)
    
        df_sub['Funding Amount - ITC (Snapshot)'].ffill(inplace=True)
        df_sub['Backlog NR'].ffill(inplace=True)
        df_sub['Forecast Direct Labor Cost - ITC'].ffill(inplace=True)
        df_sub['Forecast Net Revenue - ITC'].ffill(inplace=True)
        df_sub['Baseline NR'].bfill(0, inplace=True)
    
    
        df_sub['Actual Direct Labor Cost - MTD'].fillna(0, inplace=True)
        df_sub['Actual Gross Revenue - MTD'].fillna(0, inplace=True)
        df_sub['Actual Net Revenue - MTD'].fillna(0, inplace=True)
        df_sub['EBITA - MTD'].fillna(0, inplace=True) 
        df_sub['Order Intake (NR) - MTD'].fillna(0, inplace=True)  
    
        df_sub['Funding Amount - ITC (Snapshot)'].fillna(0, inplace=True)
        df_sub['Backlog NR'].fillna(0, inplace=True)
        df_sub['Forecast Direct Labor Cost - ITC'].fillna(0, inplace=True)
        df_sub['Forecast Net Revenue - ITC'].fillna(0, inplace=True)
        df_sub['Baseline NR'].fillna(0, inplace=True)
    
    #df_sub['Project Progress Status (Current)'].bfill(0, inplace=True)
    
        appended_data.append(df_sub)
    
        df_master=pd.concat(appended_data)
    return df_master    

In [228]:
def removal_loop(dataframe):
    chuncks = obtain_chucklist(dataframe)
    master_df = remove_NA(chuncks, dataframe)
    return master_df

In [ ]:
dfff = removal_loop(subdf)
dfff.head()

In [262]:
def real_removal(df_sub):
        df_sub['Funding Amount - ITC (Snapshot)'].ffill(inplace=True)
        df_sub['Backlog NR'].ffill(inplace=True)
        df_sub['Forecast Direct Labor Cost - ITC'].ffill(inplace=True)
        df_sub['Forecast Net Revenue - ITC'].ffill(inplace=True)
        df_sub['Baseline NR'].bfill(0, inplace=True)
    
    
        df_sub['Actual Direct Labor Cost - MTD'].fillna(0, inplace=True)
        df_sub['Actual Gross Revenue - MTD'].fillna(0, inplace=True)
        df_sub['Actual Net Revenue - MTD'].fillna(0, inplace=True)
        df_sub['EBITA - MTD'].fillna(0, inplace=True) 
        df_sub['Order Intake (NR) - MTD'].fillna(0, inplace=True)  
    
        df_sub['Funding Amount - ITC (Snapshot)'].fillna(0, inplace=True)
        df_sub['Backlog NR'].fillna(0, inplace=True)
        df_sub['Forecast Direct Labor Cost - ITC'].fillna(0, inplace=True)
        df_sub['Forecast Net Revenue - ITC'].fillna(0, inplace=True)
        df_sub['Baseline NR'].fillna(0, inplace=True)
        
        return df_sub

In [263]:
#df_master.to_csv('df_master.csv')
df = real_removal(df)

In [264]:
#df.tail(50)

In [265]:
#df_master.head()

In [266]:
len(df)

1269171

In [17]:
#len(df_master)

### Doublcheck the following variables with tjeerd

- bookings_jtd
- interest_current
- interest_jtd
- interest_ytd
- multiplier_budget
- multiplier_current
- multiplier_target
- overhead_current
- overhead_rate
- POC


### Let's calcualte percentage of completion

In [267]:
def create_completion(df):
    df['ActualNR_sum'].ffill(inplace=True)
    df['ActualNR_sum'] = df['ActualNR_sum'].astype('int')
    df['Funding Amount - ITC (Snapshot)'] = df['Funding Amount - ITC (Snapshot)'].astype('int')
    df['Completion'] = (df['ActualNR_sum']/df['Funding Amount - ITC (Snapshot)'])
    df['Completion'].ffill(inplace=True)
    return df

In [268]:
df = create_completion(df)

In [269]:
#df_master = create_completion(df_master)

In [270]:
#df_master.head()

In [271]:
#df_master[df_master['Project Number'] == '30008040']['Funding Amount - ITC (Snapshot)']

### Include a logtransform column of Actual net revenue

In [272]:
import numpy as np

In [273]:
#df['Actual Net Revenue - MTD'] = df['Actual Net Revenue - MTD'].fillna(0.0, inplace=True) #method='ffill'
def create_logscale(df):
    df['logtransform'] = np.log(df['Actual Net Revenue - MTD']+1)
    return df

In [274]:
df=create_logscale(df)
df['logtransform'].fillna(0, inplace=True)
df['logtransform'].head()

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/julia/opt/anaconda3/lib/python3.7/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


0    4.724907
1    9.250277
2    0.000000
3    4.477564
4    5.226929
Name: logtransform, dtype: float64

In [80]:
df.columns[df.isnull().any()]

Index(['Project Number_x', 'Project Name', 'Health and Safety Status',
       'Project Status', 'SalesCloudClientNo', 'Contract Type Name',
       'Ultimate Customer AUS Market Sector', 'Division Short Name',
       'location_2', 'location_4', 'Region', 'City', 'Country', 'Country Name',
       'Proj_CVP', 'Proj_Serv_Offering', 'CVP Description', '﻿CVP Code',
       'primary_client_industry', 'Sales Cloud Opportunity Number',
       'Actual Non-Labor Cost - MTD', 'bookings_jtd', 'interest_current',
       'interest_jtd', 'interest_ytd', 'multiplier_budget',
       'multiplier_current', 'multiplier_target', 'overhead_current',
       'overhead_rate', 'revenue_type', 'AUS Mkt Sector Aggregate',
       'Break Even Multiplier - Project', 'Risk Profile', 'riskrating_project',
       'Total Aging', 'Cleaned_Name', 'Project Start Date',
       'Project Manager Employee Number (Current)', 'Scheduled End Date'],
      dtype='object')

### Include column for scheduled duration

In [275]:
import re
from datetime import datetime
from pandas import datetime

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [276]:
## The Project start date was not recorded correctly, so we had to do some transformations here before we can transform it into a proper datetime
def create_datetime(df):
    df['Project Start Date'] = df['Project Start Date'].astype(str)

    df['Project Start Date'] = df['Project Start Date'].apply(lambda x: str(x)[:10])

    df['Project Start Date'] = df['Project Start Date'].apply(lambda x: re.sub(r'19-','2019-', str(x)))
    
    df['Project Start Date'] = df['Project Start Date'].str.findall(r'\d{4}[-/]\d{2}[-/]\d{2}')
    
    df['Project Start Date'] = df['Project Start Date'].apply(lambda x: ','.join(map(str, x)))
    
    df['Project Start Date'] = pd.to_datetime(df['Project Start Date'], format='%Y-%m-%d')
    
    return df

In [277]:
df.pop('Project Number_x')

0          000221760018
1          000221760018
2          000221760018
3          000221760018
4          000221760018
               ...     
1269166    zz000zzznt20
1269167    zz000zzznt20
1269168    zz000zzznt20
1269169    zz000zzznt20
1269170    zz000zzznt20
Name: Project Number_x, Length: 1269171, dtype: object

In [278]:
df = create_datetime(df)

In [279]:
#df[df['Project Number'] == '30011431']

In [280]:
#Then we also transfrom the scheduled end date into a proper 
df['Scheduled End Date'] = pd.to_datetime(df['Scheduled End Date'], format='%Y-%m-%d') 

### now let's create a column that calculates the scheduled duration

In [281]:
df['scheduled_duration'] = df['Scheduled End Date']-df['Project Start Date']

In [282]:
df['Scheduled_projectduration'] = (df['Scheduled End Date']-df['Project Start Date']) / np.timedelta64(1, 'M')

In [283]:
df['Scheduled_projectduration']

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
           ..
1269166   NaN
1269167   NaN
1269168   NaN
1269169   NaN
1269170   NaN
Name: Scheduled_projectduration, Length: 1269171, dtype: float64

### Let's create a binary column that says whether the scheduled duration equals the actual duration

In [284]:
df.Projectduration.astype(float)

0          20.0
1          20.0
2          20.0
3          20.0
4          20.0
           ... 
1269166     6.0
1269167     6.0
1269168     6.0
1269169     6.0
1269170     6.0
Name: Projectduration, Length: 1269171, dtype: float64

In [111]:
df.columns

Index(['index', 'Project Name', 'Health and Safety Status', 'Project Status',
       'SalesCloudClientNo', 'Contract Type Name',
       'Ultimate Customer AUS Market Sector', 'Division Short Name',
       'location_2', 'location_4', 'Region', 'City', 'Country', 'Country Name',
       'Proj_CVP', 'Proj_Serv_Offering', 'CVP Description', '﻿CVP Code',
       'primary_client_industry', 'Sales Cloud Opportunity Number',
       'Funding Amount - ITC (Snapshot)', 'Backlog NR',
       'Actual Direct Labor Cost - MTD', 'Actual Non-Labor Cost - MTD',
       'Actual Gross Revenue - MTD', 'Actual Net Revenue - MTD', 'EBITA - MTD',
       'Forecast Direct Labor Cost - ITC', 'Forecast Net Revenue - ITC',
       'Order Intake (NR) - MTD', 'bookings_jtd', 'interest_current',
       'interest_jtd', 'interest_ytd', 'multiplier_budget',
       'multiplier_current', 'multiplier_target', 'overhead_current',
       'overhead_rate', 'revenue_type', 'AUS Mkt Sector Aggregate',
       'Baseline NR', 'Break Eve

In [285]:
import math

In [286]:
df['Scheduled_projectduration'].fillna(0, inplace=True)
df['intime'] = 0
df['round']=df['Scheduled_projectduration'].apply(lambda x: math.ceil(x))

In [292]:
df['round']

0          0
1          0
2          0
3          0
4          0
          ..
1269166    0
1269167    0
1269168    0
1269169    0
1269170    0
Name: round, Length: 1269171, dtype: int64

In [306]:
for i,row in df.iterrows():
    if row['round'] == row['Projectduration']:
        df.at[i,'intime'] += 1        

In [307]:
df['intime'].value_counts()

0    1242075
1      27096
Name: intime, dtype: int64

In [290]:
len(df) , df[df['intime'] == 0].count()

1269171

In [129]:
df.pop('round')
df.pop('intime')

0          0
1          0
2          0
3          0
4          0
          ..
1269166    0
1269167    0
1269168    0
1269169    0
1269170    0
Name: intime, Length: 1269171, dtype: int64

### Let's create a seasonality column

In [151]:
df.Date.astype('str') 

0          201711
1          201712
2          201801
3          201802
4          201803
            ...  
1269166    201902
1269167    201903
1269168    201904
1269169    201905
1269170    201906
Name: Date, Length: 1269171, dtype: object

In [301]:
df['month'] = df['Date'].apply(lambda x: str(x)[-2:])

In [302]:
df['month']=df['month'].astype('int') 

In [303]:
df['seasonality_month'] = 0

In [304]:
for index in range(len(df)):
    if df['month'].iloc[index] > 0 and df['month'].iloc[index] < 4:
        df['seasonality_month'].iloc[index] += 1
    if df['month'].iloc[index] > 3 and  df['month'].iloc[index] < 7:  
        df['seasonality_month'].iloc[index] += 2
    if df['month'].iloc[index] > 6 and df['month'].iloc[index] < 9 :  
        df['seasonality_month'].iloc[index] += 3  
    if df['month'].iloc[index] > 8:
        df['seasonality_month'].iloc[index] += 4             

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [305]:
df['seasonality_month'].value_counts()

1    415358
4    394021
2    284301
3    175491
Name: seasonality_month, dtype: int64

In [137]:
df.to_csv('mv_port_processed.csv')

In [8]:
import pandas as pd
import numpy as np

In [87]:
df = pd.read_csv('mv_port_processed.csv')

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (2,3,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,40,41,45,48,51,53,55,58) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [88]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df['Project Number'] = df['Project Number'].astype('str') 
df = df.loc[:, ~df.columns.str.contains('^index')]
df['scheduled_duration'] = df['scheduled_duration'].str.replace(r'days', '')
df['scheduled_duration'] = df['scheduled_duration'].replace(np.nan, 0)
df['scheduled_duration'] = df['scheduled_duration'].astype('int') 

In [40]:
df.iloc[:, :30].isna().sum()

Project Name                            168475
Health and Safety Status                428481
Project Status                          168459
SalesCloudClientNo                      329554
Contract Type Name                      168459
Ultimate Customer AUS Market Sector     319033
Division Short Name                     168381
location_2                              327900
location_4                              327900
Region                                  269896
City                                    265135
Country                                 168381
Country Name                            252561
Proj_CVP                                555006
Proj_Serv_Offering                      512103
CVP Description                         168381
﻿CVP Code                              1051445
primary_client_industry                 168381
Sales Cloud Opportunity Number          428576
Funding Amount - ITC (Snapshot)              0
Backlog NR                                   0
Actual Direct

In [41]:
df.iloc[:, 31:].isna().sum()

interest_jtd                                  823884
interest_ytd                                  823884
multiplier_budget                             823884
multiplier_current                            823884
multiplier_target                             823884
overhead_current                              823884
overhead_rate                                 823884
revenue_type                                  462571
AUS Mkt Sector Aggregate                      319055
Baseline NR                                        0
Break Even Multiplier - Project               266524
Organization Size                                  0
Risk Profile                                  168381
riskrating_project                            752428
Total Aging                                  1269171
Cleaned_Name                                  168437
ActualNR_sum                                       0
Projectduration                                    0
Project Start Date                            

In [89]:
'''Project Name'''

df['Project Name'].fillna('Unspecified', inplace=True)
df['Project Name'].unique()

array(['Univar Richmond Ph 2 Environ', 'Univar Ft Wayne IN 2018',
       'Univar Odessa TX Branch 2018 A', ..., 'NCA 76 351664 Year 3',
       'SCA 76 351801 Year 3', 'SCA 76 351828 Year 3'], dtype=object)

In [90]:
''' Health and Safety Status '''

df['Health and Safety Status'].fillna('Plan Required', inplace=True)
df['Health and Safety Status'].unique()

array(['Plan Required', 'Plan Signed Off', 'Plan Not Required',
       'Unspecified'], dtype=object)

In [91]:
''' Project Status '''

df['Project Status'].fillna('Unspecified', inplace=True)
df['Project Status'].unique()

array(['Open/Active', 'On Hold - Inactive', 'Closed',
       'Closed-Open to Collect', 'Final Closure Requested', 'Unspecified',
       'New - Pending Approval', 'Rejected', 'New', 'Manual_Adj'],
      dtype=object)

In [120]:
'''SalesCloudClientNo '''

df['SalesCloudClientNo'] = df['SalesCloudClientNo'].replace(np.nan, 0)

In [93]:
''' Contract Tyoe Name -  we exclude some here '''

df = df[df['Contract Type Name'] != 'R']
df = df[df['Contract Type Name'] != 'N']
df = df[df['Contract Type Name'] != 'D']
df['Contract Type Name'].fillna('Unspecified', inplace=True)

df['Contract Type Name'].unique()

array(['Time and Material with cap', 'Unspecified', 'Lump Sum',
       'Time and Material', 'Cost plus Fixed Fee', 'Lump Sum with IFRS',
       'Revenue group', 'H'], dtype=object)

In [94]:
''' Ultimate Customer AUS Market Sector  '''

df['Ultimate Customer AUS Market Sector'].fillna('Unspecified', inplace=True)
df['Ultimate Customer AUS Market Sector'].unique()

array(['Unspecified', 'Oil & Gas', 'Municipalities/Local Govt.',
       'Water Companies/Utilities', 'State/Regional Government',
       'Chemicals', 'Life Sciences', 'Intercompany',
       'Commercial Developers', 'Aerospace', 'Consumer Goods & Retail',
       'Power Companies/Utilities', 'Conglomerates',
       'Institutional Agencies', 'Rail', 'Central/Federal Government',
       'Mining & Metals', 'Technology', 'Automotive',
       'Ports/Transportation', 'Education', 'Other Private',
       'Fin. Institutions/Investors', 'Healthcare', 'Non-Rail'],
      dtype=object)

In [95]:
''' Division Short Name   '''

df['Division Short Name'].fillna('Unspecified', inplace=True)
df['Division Short Name'].unique()

array(['Environment', 'Water', 'Infrastructure', 'Enabling Function',
       'Buildings', 'Unspecified'], dtype=object)

In [96]:
''' location_4, location_2   '''

df['location_4'].fillna('Unspecified', inplace=True)
df['location_2'].fillna('Unspecified', inplace=True)
df['location_2'].unique()
df['location_4'].unique()

array(['NT', 'WE', 'PY', 'FL', 'NJ', 'TN', 'CH', 'VA', 'ID', 'OO', 'EA',
       'DM', 'IP', 'NG', 'ED', 'SN', 'NY', 'AE', 'FS', 'CS', 'WA', 'CW',
       'TE', 'WT', 'CF', 'IS', 'HC', 'CI', 'SW', 'CT', 'PD', 'WI',
       'Unspecified', 'OE', 'SE', 'PW', 'AF', 'CD', 'AN', 'DE', 'SL',
       'SK', 'CA', 'RR', 'BN', 'PE', 'CC', 'AW', 'LC', 'AK', 'OH', 'IM',
       'ND', 'BH', 'ET', 'TI', 'CO', 'BS', 'OG', 'TC', 'PH', 'CG', 'FC',
       'TP', 'MS', 'GA', 'TS', 'GS', 'NP', 'LA', 'CJ', 'NO', 'TX', 'WR',
       'CP', 'CB', 'PA', 'AZ', 'EN', 'SD', 'HR', 'TK', 'SA', 'AD', 'DI',
       'OS', 'WV', 'PS', 'PK', 'WM', 'AC', 'IN', 'KW', 'I1', 'IT', 'MC',
       'BU', 'CE', 'CU', 'LG', 'NV', 'PX', 'OT', 'NM', 'TD', 'TM', 'I5',
       'ST', 'CQ', 'HB', 'HM', 'HS', 'I2', 'I3', 'I6', 'JA', 'KC', 'KS',
       'KN', 'JO', 'JP', 'KO', 'PL', 'PT', 'SS', 'TZ', 'WW', 'DD', 'PM',
       'EV', 'FR', 'PN', 'JE', 'HA', 'HZ', 'MA', 'MP', 'GB', 'GE', 'GI',
       'GW', 'IC', 'PO', 'I7', 'HP'], dtype=object)

In [97]:
'''Region '''

df['Region'].fillna('Unspecified', inplace=True)
df['Region'].unique()

array(['British Columbia, CA', 'Indiana, US', 'Texas, US',
       'Washington, US', 'California, US', 'West Virginia, US',
       'Pennsylvania, US', 'New York, US', 'Florida, US',
       'New Jersey, US', 'AR', 'Michigan, US', 'NY', 'Virginia, US',
       'Ohio, US', 'OH', 'MD', 'Maryland, US', 'DC',
       'District of Columbia, US', 'TX', 'Puerto Rico, PR', 'PA', 'CT',
       'Massachusetts, US', 'Unspecified', 'Delaware, US',
       'North Carolina, US', 'Connecticut, US', 'Arizona, US', 'VA', 'NJ',
       'AZ', 'Louisiana, US', 'FL', 'Missouri, US', 'CA', 'Oregon, US',
       'Alabama, US', 'Colorado, US', 'NM', 'Georgia, US',
       'North Dakota, US', 'Minnesota, US', 'IN', 'NC', 'MI',
       'Mississippi, US', 'IL', 'Iowa, US', 'Wisconsin, US', 'DE',
       'Kentucky, US', 'MS', 'MAH', 'WI', 'IA', 'Oklahoma, US', 'MO',
       'Alaska, US', 'GA', 'CO', 'Kansas, US', 'Illinois, US',
       'New Hampshire, US', 'TN', 'Tennessee, US', 'Alberta, CA',
       'Ontario, CA', 'Maine, US

In [127]:
''' CVP code will be dropped'''
df=df.drop(df.columns[14], axis=1)

In [122]:
'''Country '''

df['Country'].fillna('Unspecified', inplace=True)
df['Country'].unique()

array(['Canada', 'United States', 'Germany', 'Brazil', 'Mexico', 'India',
       'Maldives', 'Haiti', 'Unspecified', 'Saudi Arabia', 'Italy',
       'Saint Lucia', 'Chile', 'Australia', 'Bahamas', 'United Kingdom',
       'Uruguay', 'Ethiopia', 'Peru', 'Greenland', 'Ireland', 'China',
       'Puerto Rico', 'Israel', 'Colombia', 'Belgium', 'Netherlands',
       'Guatemala', 'Honduras', 'Korea, Republic of', 'Aruba', 'Japan',
       'Sweden', 'Hong Kong', 'Singapore', 'Taiwan',
       'United Arab Emirates', 'South Africa', 'Argentina', 'France',
       'Thailand', 'Philippines', 'Indonesia', 'Panama', 'El Salvador',
       'Nicaragua', 'Ecuador', 'Bermuda', 'Norway', 'New Zealand',
       'Finland', 'Denmark', 'Switzerland', 'Estonia', 'Malaysia',
       'Hungary', 'Virgin Islands (US)', 'Morocco', 'Spain', 'Belize',
       'Cambodia', 'Czech Republic', 'Bolivia', 'Russian Federation',
       'Austria', 'Cayman Islands', 'Qatar', 'Afghanistan',
       'Dominican Republic', 'Egypt'], dty

In [ ]:
df.pop('City')
df.pop('Country Name')

In [101]:
'''Proj_CVP '''

df['Proj_CVP'].fillna('Unspecified', inplace=True)
df['Proj_CVP'] = df['Proj_CVP'].str.replace(r'N/A - NOT APPLICABLE', 'Unspecified')
df['Proj_CVP'].unique()

array(['Environmental Planning', 'Site Evaluation and Restoration',
       'Water for Industry', 'Conveyance', 'Water Supply and Treatment',
       'Water Management', 'Business Advisory',
       'Strategic Environmental Consulting', 'Unspecified',
       'Environmental Construction Services', 'Program Management (PM)',
       'Ports and Industrial Infrastructure Solutions',
       'Rail & Urban Transport',
       'Master Planning & Sustainable Urban Development',
       'Cost Management (CM)', 'Mining Solutions',
       'Performance Driven Engineering (PDE)', 'Contract Solutions',
       'Commercially Led Program Management', 'GRIP', 'Highways & ITS',
       'Performance Driven Architecture (PDA)', 'Airport Solutions',
       'Performance Driven Design', 'Overhead Project'], dtype=object)

In [102]:
'''Proj_Serv_Offering '''

df['Proj_Serv_Offering'].fillna('Unspecified', inplace=True)
df['Proj_Serv_Offering'] = df['Proj_Serv_Offering'].str.replace(r'N/A - NOT APPLICABLE', 'Unspecified')

In [103]:
'''Proj_Serv_Offering '''

df['CVP Description'].fillna('Unspecified', inplace=True)
#df['Proj_Serv_Offering'] = df['Proj_Serv_Offering'].str.replace(r'N/A - NOT APPLICABLE', 'Unspecified')
df['CVP Description'].unique()

array(['Environmental Planning', 'Site Evaluation and Restoration',
       'Water for Industry', 'Conveyance', 'Water Supply and Treatment',
       'Unspecified', 'Water Management', 'Business Advisory',
       'Strategic Environmental Consulting',
       'Environmental Construction Services',
       'Project and Program Management',
       'Ports and Industrial Infrastructure Solutions',
       'Master Planning & Sustainable Urban Development',
       'Rail & Urban Transport Solutions', 'Default',
       'Cost and Commercial Management', 'Mining Solutions',
       'Performance Driven Engineering', 'Contract Solutions',
       'Highways and ITS Solutions', 'Performance Driven Architecture',
       'GRIP', 'Airport Solutions', 'Overhead Project'], dtype=object)

In [104]:
''' primary_client_industry '''

df['primary_client_industry'].fillna('Unspecified', inplace=True)
#df['Proj_Serv_Offering'] = df['Proj_Serv_Offering'].str.replace(r'N/A - NOT APPLICABLE', 'Unspecified')
df['primary_client_industry'].unique()

array(['CP - Chem/Pharma', 'OG - Oil & Gas', 'MU - Municipal',
       'WC - Water Companies/Utilities', 'ST - State/Provinces',
       'CONG - Conglomerate', 'PPA - Ports/Public Authority',
       'EE - Engineering, Environmental & Construction Management',
       'TS - Transit', 'AE - Aerospace', 'PC - Power Companies/Utilities',
       'PA - Pulp/Paper', 'NP - Non-Profit', 'RL - Freight Rail',
       'FED - Federal',
       'BLD - Building Materials & General Manufacturing',
       'CR - Consumer & Retail', 'DOD - Dept of Defense',
       'MM - Mining/Metals', 'F&BP - Food & Bev Producers',
       'CT - Contractors', 'TC - Telecom Utilities',
       'EL - Technology, Telecomm & Electronics', 'FIN - Financial',
       'CD - Commercial Developers', 'PubEd - Public Education',
       'AU - Automotive', 'IN - Internal', 'DOT - State Transportation',
       'IC - Arcadis Intercompany', 'DC - Digital & Creative Industries',
       'PvtEd - Private Education', 'Unspecified', 'INS - Insuranc

In [105]:
df['Sales Cloud Opportunity Number'] = df['Sales Cloud Opportunity Number'].replace(np.nan, 0)

In [106]:
df['Actual Non-Labor Cost - MTD'] = df['Actual Non-Labor Cost - MTD'].replace(np.nan, 0)

In [129]:
df['bookings_jtd'] = df['bookings_jtd'].replace(np.nan, 0)
df['interest_current'] = df['interest_current'].replace(np.nan, 0)
df['interest_jtd'] = df['interest_jtd'].replace(np.nan, 0)
df['interest_ytd'] = df['interest_ytd'].replace(np.nan, 0)
df['multiplier_budget'] = df['multiplier_budget'].replace(np.nan, 0)
df['multiplier_current'] = df['multiplier_current'].replace(np.nan, 0)

df['multiplier_target'] = df['multiplier_target'].replace(np.nan, 0)
df['overhead_current'] = df['overhead_current'].replace(np.nan, 0)

df['overhead_rate'] = df['overhead_rate'].replace(np.nan, 0)
df['overhead_current'] = df['overhead_current'].replace(np.nan, 0)


In [109]:
''' primary_client_industry '''

df['revenue_type'].fillna('H', inplace=True)
df['revenue_type'].unique()

array(['I', 'G', 'N', 'R', 'B', 'H', 'C', '3'], dtype=object)

In [110]:
df.groupby(['revenue_type']).size()

revenue_type
3      1168
B     91945
C       899
G    287756
H    462634
I    417067
N      5355
R       153
dtype: int64

In [112]:
''' AUS Mkt Sector Aggregate '''

df['AUS Mkt Sector Aggregate'].fillna('Other', inplace=True)
df['AUS Mkt Sector Aggregate'].unique()

array(['Other', 'Oil & Gas', 'Public', 'Chemicals', 'Life Sci',
       'Intercompany', 'Commercials', 'Manufacturing', 'Power/Util',
       'Transp/Log', 'Federal', 'Mng/Metals'], dtype=object)

In [113]:
df['Break Even Multiplier - Project'] = df['Break Even Multiplier - Project'].replace(np.nan, 0)

In [115]:
'''Risk profile'''

df['Risk Profile'].fillna('Unspecified', inplace=True)
df['Risk Profile'].unique()

array(['Unspecified', 'High', 'Low', 'Medium'], dtype=object)

In [117]:
'''Risk profile'''

df['riskrating_project'] = df['riskrating_project'].replace(np.nan, 0)
df['riskrating_project'].unique()

array([0., 1., 2., 3., 4., 5.])

In [118]:
df.pop('Total Aging')
df.pop('Cleaned_Name')
df.pop('Project Manager Employee Number (Current)')

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
           ..
1269166   NaN
1269167   NaN
1269168   NaN
1269169   NaN
1269170   NaN
Name: Project Manager Employee Number (Current), Length: 1266977, dtype: float64

In [130]:
df.iloc[:, :30].isna().sum()

Project Name                           0
Health and Safety Status               0
Project Status                         0
SalesCloudClientNo                     0
Contract Type Name                     0
Ultimate Customer AUS Market Sector    0
Division Short Name                    0
location_2                             0
location_4                             0
Region                                 0
Country                                0
Proj_CVP                               0
Proj_Serv_Offering                     0
CVP Description                        0
primary_client_industry                0
Sales Cloud Opportunity Number         0
Funding Amount - ITC (Snapshot)        0
Backlog NR                             0
Actual Direct Labor Cost - MTD         0
Actual Non-Labor Cost - MTD            0
Actual Gross Revenue - MTD             0
Actual Net Revenue - MTD               0
EBITA - MTD                            0
Forecast Direct Labor Cost - ITC       0
Forecast Net Rev

In [131]:
df.iloc[:, 31:].isna().sum()

multiplier_current                      0
multiplier_target                       0
overhead_current                        0
overhead_rate                           0
revenue_type                            0
AUS Mkt Sector Aggregate                0
Baseline NR                             0
Break Even Multiplier - Project         0
Organization Size                       0
Risk Profile                            0
riskrating_project                      0
ActualNR_sum                            0
Projectduration                         0
Project Start Date                 259785
Scheduled End Date                 260107
Date                                    0
Project Number                          0
Completion                              0
logtransform                            0
scheduled_duration                      0
Scheduled_projectduration               0
intime                                  0
round                                   0
month                             

In [132]:
df.pop('Project Start Date')
df.pop('Scheduled End Date')

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
          ... 
1269166    NaN
1269167    NaN
1269168    NaN
1269169    NaN
1269170    NaN
Name: Scheduled End Date, Length: 1266977, dtype: object

In [139]:
'''Organization Size '''


df['Organization Size'] = df['Organization Size'].str.replace(r'GKC', 'Key Global')
df['Organization Size'] = df['Organization Size'].str.replace(r'Key Local', 'Local')
df['Organization Size'] = df['Organization Size'].str.replace(r'RKC', 'Key Regional')


df['Organization Size'].unique()

array(['Key Regional', 'Local', 'Key Global', 'Unspecified'], dtype=object)

In [140]:
df.to_csv('mv_port_processed2.csv')